In [ ]:
print("Hello World")

In [ ]:
import pyodbc

# ----------------------------
# STEP 1: Define your connection
# ----------------------------
server = r'(localdb)\MSSQLLocalDB'    # local SQL Server instance
database = 'Arsipa'             # make sure the name matches exactly

# define conn_str for Windows Authentication for SQL Server

conn_str = (
    "Driver={ODBC Driver 17 for SQL Server};"
    f"Server={server};"
    f"Database={database};"
    "Trusted_Connection=yes;"
    "Encrypt=no;"
)

# ----------------------------
# STEP 2: Connect to SQL Server
# ----------------------------
try:
    conn = pyodbc.connect(conn_str)
    print("✅ Connection to SQL Server successful!")
except Exception as e:
    print("❌ Failed to connect to SQL Server:")
    print(e)
    conn = None

# ----------------------------
# STEP 3: Run a simple query
# ----------------------------
if conn:
    try:
        cursor = conn.cursor()
        cursor.execute("SELECT @@VERSION;")  # Returns SQL Server version
        for row in cursor.fetchall():
            print("SQL Server Version:", row[0])
    except Exception as e:
        print("❌ Query failed:", e)
    finally:
        conn.close()
        print("🔒 Connection closed.")


In [ ]:
import pandas as pd

# Load CSV Data files with encoding
df_umsatz = pd.read_csv(r"C:\Users\Dell\Desktop\Python Practice\source_datafiles\csv\umsatz.csv", encoding="utf-8")
df_mitarbeiter = pd.read_csv(r"C:\Users\Dell\Desktop\Python Practice\source_datafiles\csv\mitarbeiter.csv", encoding="utf-8")
df_gesellschaften = pd.read_csv(r"C:\Users\Dell\Desktop\Python Practice\source_datafiles\csv\gesellschaften.csv", encoding="latin1")

# View Datasets
print("📊 Umsatz (erste 5 Zeilen):")
print(df_umsatz.head(5))

print("\n👨‍💼 Mitarbeiter (letzte 5 Zeilen):")
print(df_mitarbeiter.tail(5))

print("\n🏢 Gesellschaften (erste 5 Zeilen):")
print(df_gesellschaften.head(5))

In [ ]:
# Joining Fact Table with Dim(s)
fact = ( df_umsatz.merge(df_mitarbeiter, on=["gesellschaft_id", "monat"], how="left")
                                        .merge(df_gesellschaften, on="gesellschaft_id", how="left")
       )

# Example: KPI "Sales_per_Employee"
fact["umsatz_pro_mitarbeiter"] = fact["umsatz_eur"] / fact["anzahl_mitarbeiter"].replace(0, pd.NA)

# View Fact Table after Joins
print("Fact Table after Joins\n", fact.head(5))

# Group by gesellschaft_id and calculate SUM of umsatz_pro_mitarbeiter
result_sum = fact.groupby("gesellschaft_id")["umsatz_pro_mitarbeiter"].sum()

# Group by gesellschaft_id and calculate MEAN (average)
result_mean = fact.groupby(["gesellschaft_id", "monat"])["umsatz_pro_mitarbeiter"].mean()

# If you want both at once
result = fact.groupby("gesellschaft_id")["umsatz_pro_mitarbeiter"].agg(["sum", "mean", "max", "min"])

print("MEANS Table\n", result_mean.head(5))
print("Caclucations Table Grouped by Gesellschat&ProMitarbeiter\n",result.head(5))

